### 교차 검증(Cross Validation)

#### 테스트 데이터에만 과적합 될 수 있으므로 데이터를 여러개로 나누어 테스트를 여러번 수행하여 평균 정확도를 구함

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score,classification_report

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
iris_data = load_iris()

# train(학습) 데이터와 test(검증) 데이터 세트로 분리 : 80%:20%, 120개(train),30개(test)
X_train,X_test,y_train, y_test = train_test_split(iris_data.data, iris_data.target,
                                                  test_size=0.2,
                                                  random_state=11)  # random seed를 고정
dt_clf = DecisionTreeClassifier(random_state=11)

# 학습
dt_clf.fit(X_train,y_train)

# 예측
pred = dt_clf.predict(X_test)  # y값 예측

#정확도 측정 : 1회만 예측한 결과의 정확도
print('정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

# -------------------------------------------------------------------------
# 교차 검증(cross validation)
# cross_val_score에 들어가는 매개변수는 (모델 명, 훈련데이터, 타깃, cv) 
cv_score = cross_val_score(dt_clf,iris_data.data,iris_data.target,
                           scoring='accuracy',cv=3) # 3개로 쪼개어 검증
# 데이터셋을 3개로 쪼개어 검증(fit,predict를 3회 수행)
# 내부적으로 Stratified K-Fold가 사용됨, 평가지표를 하나만 구할 수 있어서 
# StratifiedKFold 사용 권장

print('교차 검증 정확도:',cv_score)
print('교차 검증 평균 정확도:',cv_score.mean())
# -------------------------------------------------------------------------

정확도: 0.9333
교차 검증 정확도: [0.98 0.92 0.98]
교차 검증 평균 정확도: 0.96


### K-Fold 교차 검증
#### : K번 만큼 폴드(Fold)된 각각의 데이터 세트로 학습과 검증을 K번 만큼 반복하여 평균 정확도를 구함

In [12]:
from sklearn.model_selection import KFold
iris = load_iris()
features = iris.data  # X, 피쳐
label = iris.target   # Y, 레이블

features.shape  # (150,4)
label.shape     # (150,)

dt_clf = DecisionTreeClassifier(random_state=11)  # Estimator 생성

In [29]:
# 5개 폴드 세트로 분리
kfold = KFold(n_splits = 5)   # class의 instance생성
cv_accuracy = []  # 교차 검증 정확도 리스트
n_iter = 0  # 반복횟수

for train_index,test_index in kfold.split(features):
    # kfold.split()함수 : X데이터 features를 분리하여 학습/검증용 데이터의 인덱스를 반환


    # 150개 데이터
    # k값(n_splits) : 폴드 세트 갯수,   학습:검증    검증 데이터의 비율
    #       2       :        2           75:75         1/2 (50%)
    #       3       :        3          100:50         1/3 (33%)
    #       4       :        4          112:38         1/4 (25%)
    #       5       :        5          120:30         1/5 (20%)   <- 여기서 쓰는 것
    #       6       :        6          125:25         1/6 (16.7%)
#    print(train_index) # test-train-train-train-train / train-test-train-train-train ...
#    print(test_index)  # index값들임
    
    X_train = features[train_index]
    X_test = features[test_index]
    
    y_train = label[train_index]
    y_test = label[test_index]
    
    # 학습 및 예측
    dt_clf.fit(X_train,y_train)
    pred = dt_clf.predict(X_test)
    
    n_iter +=1
    
    # 반복시마다 정확도 측정
    accuracy = round(accuracy_score(y_test,pred),4)
    cv_accuracy.append(accuracy)
    
    print('#',n_iter,'번 폴드 교차검증 정확도:',accuracy,
          '학습 데이터의 크기:',X_train.shape[0],
          '검증 데이터의 크기:',X_test.shape[0])
    
    print('학습 데이터의 인덱스:\n',train_index)
    print('검증 데이터의 인덱스:\n',test_index)
    
print('K폴드 교차 검증 평균 정확도:',round(np.mean(cv_accuracy),4))  # 0.92

# 1 번 폴드 교차검증 정확도: 1.0 학습 데이터의 크기: 120 검증 데이터의 크기: 30
학습 데이터의 인덱스:
 [ 30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47
  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]
검증 데이터의 인덱스:
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
# 2 번 폴드 교차검증 정확도: 0.9667 학습 데이터의 크기: 120 검증 데이터의 크기: 30
학습 데이터의 인덱스:
 [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  9

### Stratified K-Fold : 필수 사용!!
#### label이 지나치게 불균형 분포를 이룰 때 레이블의 분포를 균일하게 폴드시키는 방식
#### 분포 모델에서만 가능(회귀모델 지원되지 않음)

In [22]:
iris = load_iris()
iris_df = pd.DataFrame(data=iris.data,columns=iris.feature_names)
iris_df['label'] = iris.target
iris_df['label'].value_counts()

kfold = KFold(n_splits=5,shuffle=True)  # shuffle과 Stratified 와는 성능이 다름
n_iter = 0
for train_index, test_index in kfold.split(features):
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('\n[',n_iter,'번 폴드 세트]')
    print(label_train.value_counts())  # 120개
    print(label_test.value_counts())   # 30개, 레이블이 불균형 분포를 이룬다



[ 1 번 폴드 세트]
2    42
0    42
1    36
Name: label, dtype: int64

[ 2 번 폴드 세트]
1    46
2    40
0    34
Name: label, dtype: int64

[ 3 번 폴드 세트]
0    46
2    37
1    37
Name: label, dtype: int64

[ 4 번 폴드 세트]
2    41
1    40
0    39
Name: label, dtype: int64

[ 5 번 폴드 세트]
1    41
2    40
0    39
Name: label, dtype: int64


In [31]:
# StratifiedKFold : 레이블의 분포가 균일하게 분리된다, 정확도 향상
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits = 5)
n_iter = 0

for train_index,test_index in skf.split(iris_df,iris_df['label']):
    n_iter+=1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('\n[',n_iter,'번 폴드 세트]')
    print(label_train.value_counts())  # 120개
    print(label_test.value_counts())   # 30개
    # 원본과 같은 비율로 레이블이 균형 분포를 이룬다
    



[ 1 번 폴드 세트]
2    40
1    40
0    40
Name: label, dtype: int64
2    10
1    10
0    10
Name: label, dtype: int64

[ 2 번 폴드 세트]
2    40
1    40
0    40
Name: label, dtype: int64
2    10
1    10
0    10
Name: label, dtype: int64

[ 3 번 폴드 세트]
2    40
1    40
0    40
Name: label, dtype: int64
2    10
1    10
0    10
Name: label, dtype: int64

[ 4 번 폴드 세트]
2    40
1    40
0    40
Name: label, dtype: int64
2    10
1    10
0    10
Name: label, dtype: int64

[ 5 번 폴드 세트]
2    40
1    40
0    40
Name: label, dtype: int64
2    10
1    10
0    10
Name: label, dtype: int64


In [34]:
#  StratifiedKFold를 사용하여 학습 및 예측과 정확도 측정(회귀 모델에서는 사용불가능)
skf = StratifiedKFold(n_splits = 5)
cv_accuaracy = []
n_iter = 0

for train_index, test_index in skf.split(iris_df,iris_df['label']): # Y레이블을 반드시 인자로 사용
    X_train = features[train_index]
    X_test = features[test_index]
    
    y_train = label[train_index]
    y_test = label[test_index]
    
    # 학습 및 예측
    dt_clf.fit(X_train,y_train)
    pred = dt_clf.predict(X_test)
    
    n_iter +=1
    
    # 반복 시 마다 정확도 측정
    accuracy = round(accuracy_score(y_test,pred),4)
    cv_accuracy.append(accuracy)
    
    print('#',n_iter,'번 폴드 교차검증 정확도:',accuracy,
          '학습 데이터의 크기:',X_train.shape[0],
          '검증 데이터의 크기:',X_test.shape[0])
    
    print('학습 데이터의 인덱스:\n',train_index)
    print('검증 데이터의 인덱스:\n',test_index)

# StratifiedKFold로 반복된 정확도를 합하여 평균 정확도 계산
print('StratifiedKFold 교차 검증 평균 정확도:',round(np.mean(cv_accuracy),4))  # 0.96

# StratifiedKFold로 레이블의 분포를 원본과 같이 균일하게 폴드를 생성하여 학습시키므로
# 정확도가 향상됨 : 0.92 --> 0.96


# 1 번 폴드 교차검증 정확도: 0.9667 학습 데이터의 크기: 120 검증 데이터의 크기: 30
학습 데이터의 인덱스:
 [ 10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27
  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45
  46  47  48  49  60  61  62  63  64  65  66  67  68  69  70  71  72  73
  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91
  92  93  94  95  96  97  98  99 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]
검증 데이터의 인덱스:
 [  0   1   2   3   4   5   6   7   8   9  50  51  52  53  54  55  56  57
  58  59 100 101 102 103 104 105 106 107 108 109]
# 2 번 폴드 교차검증 정확도: 0.9667 학습 데이터의 크기: 120 검증 데이터의 크기: 30
학습 데이터의 인덱스:
 [  0   1   2   3   4   5   6   7   8   9  20  21  22  23  24  25  26  27
  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45
  46  47  48  49  50  51  52  53  54  55  56  57  58  59  70  71  72  73
  74  75  76  

### GridSearchCV
#### 교차 검증과 파라메터 집합을 만들어 주면 최적의 파라메터 값을 구해줌 (파라메터=매개변수)

In [36]:
from sklearn.model_selection import GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(iris.data,iris.target, 
                                                    test_size = 0.2,
                                                    random_state=11)
dt_clf = DecisionTreeClassifier(random_state =11) # Estimator

# 파라메터들을 dict 형태로 설정
parameters = {'max_depth':[1,2,3],'min_samples_split':[2,3] }

In [38]:
grid_tree = GridSearchCV(dt_clf,param_grid=parameters,refit=True,
                         return_train_score=True)
grid_tree.fit(X_train,y_train)

scores_df = pd.DataFrame(grid_tree.cv_results_)
scores_df[['params','mean_test_score','rank_test_score',
          'split0_train_score','split1_train_score','split2_train_score']]

,params,mean_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.675000,5,0.666667,0.677083,0.677083
1,"{'max_depth': 1, 'min_samples_split': 3}",0.675000,5,0.666667,0.677083,0.677083
2,"{'max_depth': 2, 'min_samples_split': 2}",0.950000,3,0.979167,0.979167,0.958333
3,"{'max_depth': 2, 'min_samples_split': 3}",0.950000,3,0.979167,0.979167,0.958333
4,"{'max_depth': 3, 'min_samples_split': 2}",0.958333,1,0.989583,0.989583,0.968750
5,"{'max_depth': 3, 'min_samples_split': 3}",0.958333,1,0.989583,0.989583,0.968750


In [41]:
print('GridSearchCV 최적 파라메터:',grid_tree.best_params_)
print('GridSearchCV 최고 정확도:',grid_tree.best_score_)

GridSearchCV 최적 파라메터: {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도: 0.9583333333333333


In [43]:
pred = grid_tree.predict(X_test)
print('정확도:',accuracy_score(y_test,pred))

정확도: 0.9333333333333333


In [44]:
estimator = grid_tree.best_estimator_  # DecisionTreeClassifier의 인스턴스
print(estimator)
pred = estimator.predict(X_test)
print('정확도:',accuracy_score(y_test,pred))

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=3, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=11, splitter='best')
정확도: 0.9333333333333333
